In [6]:
import os

In [7]:
%pwd

'/Users/chaitanya/Documents/ML/github/Kideny-Disease-Classification-Deep-Learning-Project-DS/research'

In [8]:
os.chdir("../")

In [9]:
%pwd

'/Users/chaitanya/Documents/ML/github/Kideny-Disease-Classification-Deep-Learning-Project-DS'

In [10]:
import dagshub
dagshub.init(repo_owner='susmiti63', repo_name='Kideny-Disease-Classification-Deep-Learning-Project-DS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS"

Repository susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS initialized!

In [11]:
import dagshub
dagshub.init(repo_owner='susmiti63', repo_name='Kideny-Disease-Classification-Deep-Learning-Project-DS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS"

Repository susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS initialized!

In [12]:
import dagshub
dagshub.init(repo_owner='susmiti63',
             repo_name='Kideny-Disease-Classification-Deep-Learning-Project-DS',
             mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS"

Repository susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS initialized!

In [12]:
import tensorflow as tf

In [13]:
model = tf.keras.models.load_model("artifacts/training/model.h5")
print(model)


In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [15]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [16]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [17]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [19]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-21 22:23:11,534: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-21 22:23:11,538: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-21 22:23:11,540: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 13s 1s/step - loss: 0.0514 - accuracy: 0.9712
[2024-08-21 22:23:24,780: INFO: common: json file saved at: scores.json]


2024/08/21 22:23:25 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/vy/lbm6tmp563jbdts82tm6d2ym0000gn/T/tmpa8l25nt8/model/data/model/assets
[2024-08-21 22:23:25,846: INFO: builder_impl: Assets written to: /var/folders/vy/lbm6tmp563jbdts82tm6d2ym0000gn/T/tmpa8l25nt8/model/data/model/assets]


2024/08/21 22:23:29 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/vy/lbm6tmp563jbdts82tm6d2ym0000gn/T/tmpa8l25nt8/model, flavor: tensorflow), fall back to return ['tensorflow==2.13.0']. Set logging level to DEBUG to see the full traceback.
/opt/anaconda3/envs/kidney/lib/python3.8/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


[2024-08-21 22:23:48,172: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2639)'))': /susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS.mlflow/api/2.0/mlflow-artifacts/artifacts/ffd9d4b3a73c4a29beb256d30c6eca0f/b529f1bb561b4bfbabddc57c1f8a90b1/artifacts/model/data/model/variables/variables.data-00000-of-00001]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/08/21 22:23:56 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
